In [114]:
import pandas as pd
import numpy as np
import torch
from fastai.tabular import *

In [ ]:
np.random.seed(1)
torch.random.manual_seed(1)

In [115]:
path = Path('..', 'data', 'raw')
credit = pd.read_csv(path / 'credit_data.csv').rename(columns=lambda c: c.lower())

In [116]:
dep_var = 'status'
cat_names = ['home', 'marital', 'records', 'job']

In [117]:
credit[cat_names] = credit[cat_names].astype('category')

In [119]:
n = len(credit)
valid_idx = np.random.choice(range(n), np.ceil(n * 0.2).astype('int'))

In [120]:
procs = [FillMissing, Categorify, Normalize]

In [121]:
data = TabularDataBunch.from_df(path, credit, dep_var, valid_idx=valid_idx, procs=procs, cat_names=cat_names)
print(data.train_ds.cont_names)

['time', 'seniority', 'debt', 'age', 'amount', 'expenses', 'price', 'income', 'assets']


In [122]:
learn = tabular_learner(data, layers=[200, 100], metrics=accuracy)
learn.fit_one_cycle(5, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.593858,0.465332,0.792368,00:00
1,0.493189,0.465681,0.792368,00:00
2,0.453767,0.460289,0.789001,00:00
3,0.422882,0.417993,0.813693,00:00
4,0.406827,0.411693,0.814815,00:00


In [151]:
teste = list(learn.model.embeds.parameters())

In [152]:
teste

[Parameter containing:
 tensor([[ 0.1606,  0.1706, -0.1244, -0.0404, -0.1290],
         [ 0.1305,  0.1560, -0.1256, -0.0704, -0.1263],
         [ 0.1271,  0.0990, -0.0771, -0.0282, -0.1248],
         [-0.1257, -0.0594,  0.0457,  0.0534,  0.1699],
         [-0.1142, -0.0399,  0.0902,  0.0592,  0.1391],
         [ 0.0785,  0.0967, -0.0780, -0.0572, -0.0938],
         [ 0.1284, -0.1318, -0.0246, -0.0041, -0.0808]], device='cuda:0',
        requires_grad=True),
 Parameter containing:
 tensor([[-0.1891,  0.1384,  0.1728,  0.1882],
         [ 0.1260, -0.2028, -0.0711, -0.0811],
         [-0.1274,  0.0494,  0.0852,  0.0406],
         [ 0.3522, -0.1914, -0.3243, -0.2280],
         [-0.0156, -0.0357,  0.1079,  0.0532],
         [ 0.1158, -0.0245, -0.1317, -0.1620]], device='cuda:0',
        requires_grad=True),
 Parameter containing:
 tensor([[ 0.0033, -0.0067, -0.0158],
         [ 0.1814, -0.1839,  0.2102],
         [-0.2209,  0.1863, -0.2351]], device='cuda:0', requires_grad=True),
 Parameter

In [125]:
teste[0][0]

tensor([ 0.1606,  0.1706, -0.1244, -0.0404, -0.1290], device='cuda:0',
       grad_fn=<SelectBackward>)

In [126]:
credit['home'].cat.categories

Index(['ignore', 'other', 'owner', 'parents', 'priv', 'rent'], dtype='object')

In [146]:
list(learn.model.embeds.parameters())[0][0]

tensor([ 0.1606,  0.1706, -0.1244, -0.0404, -0.1290], device='cuda:0',
       grad_fn=<SelectBackward>)

In [160]:
len(credit['home'].cat.codes.unique()), list(learn.model.embeds.parameters())[0].shape

(7, torch.Size([7, 5]))

In [157]:
list(learn.model.embeds.parameters())[0][credit['home'].cat.codes + 1]

tensor([[ 0.1284, -0.1318, -0.0246, -0.0041, -0.0808],
        [ 0.1284, -0.1318, -0.0246, -0.0041, -0.0808],
        [-0.1257, -0.0594,  0.0457,  0.0534,  0.1699],
        ...,
        [-0.1257, -0.0594,  0.0457,  0.0534,  0.1699],
        [ 0.1284, -0.1318, -0.0246, -0.0041, -0.0808],
        [-0.1257, -0.0594,  0.0457,  0.0534,  0.1699]], device='cuda:0',
       grad_fn=<IndexBackward>)

In [179]:
teste = []
for idx, var in enumerate(cat_names):
    teste.append(list(learn.model.embeds.parameters())[idx][credit[var].cat.codes + 1])

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [185]:
np.concatenate([torch.Tensor.cpu(i).detach().numpy() for i in teste], axis=1).shape

(4454, 16)